In [91]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [92]:
stopwords = ["seat", "flight", "fly", "plane", "airline", "passenger", "hour", "date", "aircraft"]
#this stopwords were identified after taking a look at the most common words in the reviews

In [93]:
df = pd.read_csv("sent_skytrax.csv", index_col = 0)
df = df.iloc[:,1:]

In [94]:
df.head()

,author_review_count,rating,title,date,location,body,Seat Type,Aircraft Type,Seat Layout,Date Flown,...,Sitting Comfort,Seat/bed Width,Seat/bed Length,Seat Privacy,cleaned_body,scores,pos_score,neu_score,neg_score,compound_score
0,0.0,0.3,"""seat was not comfortable""",2019-07-22,Malaysia,With their newer A320 aircraft leg room is awf...,Economy Class,A320,3x3,July 2019,...,NaN,NaN,NaN,NaN,newer aircraft leg room awful seat comfortable...,"{'neg': 0.139, 'neu': 0.604, 'pos': 0.256, 'co...",0.256,0.604,0.139,0.3626
1,0.0,0.6,"""Check in was easy""",2019-04-06,Australia,Check in was easy and boarding I was in zone o...,Economy Class,A320,3x3,April 2019,...,NaN,NaN,NaN,NaN,check easy boarding zone think paid check kg l...,"{'neg': 0.068, 'neu': 0.705, 'pos': 0.227, 'co...",0.227,0.705,0.068,0.7430
2,5.0,0.7,"""A solid experience""",2019-04-01,Australia,"A solid experience from start to finish, espec...",Economy Class,A320,3x3,March 2019,...,NaN,NaN,NaN,NaN,solid experience start finish especially given...,"{'neg': 0.051, 'neu': 0.777, 'pos': 0.172, 'co...",0.172,0.777,0.051,0.8979
3,0.0,0.1,"""my 4yr old and I allocated different seats""",2018-01-14,Australia,I discovered upon boarding that my four-year-o...,Economy Class,A330-300,3x3x3,January 2018,...,NaN,NaN,NaN,NaN,discovered upon boarding four year old allocat...,"{'neg': 0.161, 'neu': 0.726, 'pos': 0.113, 'co...",0.113,0.726,0.161,-0.5267
4,1.0,0.7,"""space between was relatively good""",2017-08-24,Indonesia,"I got 14A seat, near emergency exit door. So t...",Economy Class,A320,3x3,August 2017,...,NaN,NaN,NaN,NaN,got seat near emergency exit different standar...,"{'neg': 0.157, 'neu': 0.66, 'pos': 0.183, 'com...",0.183,0.660,0.157,0.2960


In [95]:
reviews = [r for r in df.cleaned_body.tolist()]

In [96]:
#tokenize the phrases
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

reviews_words = list(sent_to_words(reviews))

print(reviews_words[:1])

[['newer', 'aircraft', 'leg', 'room', 'awful', 'seat', 'comfortable', 'seat', 'maximization', 'mean', 'gave', 'away', 'even', 'little', 'comfort', 'passenger']]


In [97]:
#seat can be considered as a stopword
reviews_words = [[i for i in sentence if i not in stopwords] for sentence in reviews_words]

In [98]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(reviews_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[reviews_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [99]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [100]:
#NOTE: stopwords were already removed in another phase of the data analysis, so now it is not necessary
# Form Bigrams
reviews_bigr = make_bigrams(reviews_words)

## NOTE: we choose bigrams but trigrams could be tested too

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
reviews_lem = lemmatization(reviews_bigr, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(reviews_lem[:1])

[['new', 'leg', 'room', 'awful', 'comfortable', 'maximization', 'mean', 'give', 'away', 'even', 'little', 'comfort']]


In [101]:
## Create the Dictionary and Corpus needed for Topic Modeling

# Create Dictionary
id2word = corpora.Dictionary(reviews_lem)

# Create Corpus
texts = reviews_lem

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


Corpus maps each phrase in (word_id, word_frequency)
If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [102]:
id2word[0]

'away'

In [103]:
# Human readable format of corpus (term-frequency)  for the first review!
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('away', 1),
  ('awful', 1),
  ('comfort', 1),
  ('comfortable', 1),
  ('even', 1),
  ('give', 1),
  ('leg', 1),
  ('little', 1),
  ('maximization', 1),
  ('mean', 1),
  ('new', 1),
  ('room', 1)]]

# Build the topic modeling - LDA

In [104]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [105]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"bag" + 0.018*"auckland" + 0.012*"bump" + 0.011*"course" + '
  '0.010*"carry" + 0.010*"future" + 0.010*"trolley" + 0.009*"landing" + '
  '0.009*"stop" + 0.009*"select"'),
 (1,
  '0.063*"economy" + 0.033*"fly" + 0.032*"class" + 0.022*"premium" + '
  '0.017*"experience" + 0.017*"business" + 0.017*"extra" + 0.015*"air" + '
  '0.015*"first" + 0.015*"pay"'),
 (2,
  '0.041*"ever" + 0.034*"never" + 0.029*"cramp" + 0.017*"travel" + '
  '0.016*"head" + 0.014*"tray" + 0.014*"horrible" + 0.013*"worst" + '
  '0.011*"say" + 0.011*"uncomfortable"'),
 (3,
  '0.031*"book" + 0.025*"check" + 0.021*"ask" + 0.016*"layout" + 0.014*"staff" '
  '+ 0.012*"arrive" + 0.012*"change" + 0.011*"padding" + 0.011*"pay" + '
  '0.011*"wait"'),
 (4,
  '0.043*"good" + 0.026*"comfortable" + 0.024*"time" + 0.024*"cabin" + '
  '0.022*"service" + 0.018*"use" + 0.017*"food" + 0.016*"entertainment" + '
  '0.016*"really" + 0.015*"well"'),
 (5,
  '0.032*"leg" + 0.030*"recline" + 0.026*"front" + 0.023*"room" + 0.022

## Compute Model Perplexity and Coherence Score
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
Coherence score is usually more helpful

In [106]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_lem, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.211652927391211

Coherence Score:  0.40447350738829163


# Find BEST NUMBER

In [107]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print("-----------------------------------------\n Building a model with: {} topics".format(num_topics))
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
        print("Num Topics =", num_topics, " has Coherence Value of", round(coherencemodel.get_coherence(), 4))

    return model_list, coherence_values

In [108]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=reviews_lem, start=6, limit=12, step=2)

-----------------------------------------
 Building a model with: 6 topics
Num Topics = 6  has Coherence Value of 0.4045
-----------------------------------------
 Building a model with: 8 topics
Num Topics = 8  has Coherence Value of 0.419
-----------------------------------------
 Building a model with: 10 topics
Num Topics = 10  has Coherence Value of 0.3808


In [111]:
# the score starts decreasing after 8, so we keep 8 topics

In [110]:
# Select the model and print the topics
optimal_model = model_list[1]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.039*"auckland" + 0.024*"course" + 0.023*"delay" + 0.021*"neck" + '
  '0.020*"stop" + 0.019*"lean" + 0.019*"break" + 0.018*"sore" + '
  '0.015*"beverage" + 0.012*"usual"'),
 (1,
  '0.089*"business" + 0.052*"class" + 0.030*"extremely" + 0.029*"know" + '
  '0.020*"difference" + 0.017*"usually" + 0.016*"fleet" + 0.014*"carrier" + '
  '0.013*"floor" + 0.013*"simply"'),
 (2,
  '0.039*"avoid" + 0.023*"tight" + 0.015*"check" + 0.014*"family" + '
  '0.013*"leather" + 0.012*"ticket" + 0.012*"tell" + 0.012*"hear" + '
  '0.012*"change" + 0.011*"book"'),
 (3,
  '0.030*"layout" + 0.021*"delta" + 0.020*"access" + 0.019*"boarding" + '
  '0.018*"do" + 0.017*"together" + 0.017*"elbow" + 0.014*"check" + '
  '0.013*"light" + 0.012*"gate"'),
 (4,
  '0.063*"good" + 0.048*"comfortable" + 0.026*"use" + 0.024*"entertainment" + '
  '0.024*"really" + 0.021*"new" + 0.020*"long" + 0.019*"old" + 0.019*"tv" + '
  '0.017*"time"'),
 (5,
  '0.047*"close" + 0.033*"size" + 0.024*"awful" + 0.021*"bother" + '
  '

In [112]:
# Find dominant topic in each sentence
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=reviews):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: x[1], reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

# for row in optimal_model[corpus]:
#     print(row[0][1])
#     break
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=reviews)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,6.0,0.4411,"leg, recline, front, room, back, row, space, u...",newer aircraft leg room awful seat comfortable...
1,1,6.0,0.3605,"leg, recline, front, room, back, row, space, u...",check easy boarding zone think paid check kg l...
2,2,6.0,0.3933,"leg, recline, front, room, back, row, space, u...",solid experience start finish especially given...
3,3,6.0,0.3516,"leg, recline, front, room, back, row, space, u...",discovered upon boarding four year old allocat...
4,4,6.0,0.4102,"leg, recline, front, room, back, row, space, u...",got seat near emergency exit different standar...
5,5,6.0,0.3721,"leg, recline, front, room, back, row, space, u...",seat pitch tight still survivable hour flight ...
6,6,6.0,0.4673,"leg, recline, front, room, back, row, space, u...",seat uncomfortable space seat poor hard time s...
7,7,6.0,0.5124,"leg, recline, front, room, back, row, space, u...",good choose airasia flight le hour distance se...
8,8,6.0,0.3029,"leg, recline, front, room, back, row, space, u...",allocated seat airasia flight ak nov kuala ban...
9,9,6.0,0.5855,"leg, recline, front, room, back, row, space, u...",seat recline notice great hate someone tilt se...


In [113]:
df_dominant_topic.index = df_dominant_topic.Document_No

In [114]:
df_dominant_topic = df_dominant_topic.drop("Document_No", axis = 1)

In [115]:
df_dominant_topic.head()

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
Document_No,,,,
0,6.0,0.4411,"leg, recline, front, room, back, row, space, u...",newer aircraft leg room awful seat comfortable...
1,6.0,0.3605,"leg, recline, front, room, back, row, space, u...",check easy boarding zone think paid check kg l...
2,6.0,0.3933,"leg, recline, front, room, back, row, space, u...",solid experience start finish especially given...
3,6.0,0.3516,"leg, recline, front, room, back, row, space, u...",discovered upon boarding four year old allocat...
4,6.0,0.4102,"leg, recline, front, room, back, row, space, u...",got seat near emergency exit different standar...


In [116]:
len(df_dominant_topic)

2594

In [117]:
df_dominant_topic.groupby("Dominant_Topic").Text.count()

Dominant_Topic
1.0       1
2.0       1
3.0       1
4.0     216
6.0    1817
7.0     558
Name: Text, dtype: int64

In [118]:
df_dominant_topic.to_csv("topic_nostopwords.csv")

# Find the most representative document for each topic

In [122]:
# Group top 5 sentences under each topic
sorted_df = pd.DataFrame()

sent_topics_outdf_grpd = df_dominant_topic.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sorted_df = pd.concat([sorted_df, 
                                             grp.sort_values(['Topic_Perc_Contrib'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sorted_df.reset_index(drop=True, inplace=True)

# Format
sorted_df.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sorted_df.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,1.0,0.2657,"business, class, extremely, know, difference, ...",flown number time business class long haul sho...
1,2.0,0.3370,"avoid, tight, check, family, leather, ticket, ...",flight toronto vancouver fine return flight le...
2,3.0,0.3157,"layout, delta, access, boarding, do, together,...",fairly typical layout business class boeing se...
3,4.0,0.6052,"good, comfortable, use, entertainment, really,...",seat tui boeing good using time long short hau...
4,6.0,0.6940,"leg, recline, front, room, back, row, space, u...",spent extra money elite class quite excited sa...


# Merge the two datasets

In [123]:
pd.concat([df_dominant_topic,df], axis = 1)

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,author_review_count,rating,title,date,location,body,...,Sitting Comfort,Seat/bed Width,Seat/bed Length,Seat Privacy,cleaned_body,scores,pos_score,neu_score,neg_score,compound_score
0,6.0,0.4411,"leg, recline, front, room, back, row, space, u...",newer aircraft leg room awful seat comfortable...,0.0,0.3,"""seat was not comfortable""",2019-07-22,Malaysia,With their newer A320 aircraft leg room is awf...,...,NaN,NaN,NaN,NaN,newer aircraft leg room awful seat comfortable...,"{'neg': 0.139, 'neu': 0.604, 'pos': 0.256, 'co...",0.256,0.604,0.139,0.3626
1,6.0,0.3605,"leg, recline, front, room, back, row, space, u...",check easy boarding zone think paid check kg l...,0.0,0.6,"""Check in was easy""",2019-04-06,Australia,Check in was easy and boarding I was in zone o...,...,NaN,NaN,NaN,NaN,check easy boarding zone think paid check kg l...,"{'neg': 0.068, 'neu': 0.705, 'pos': 0.227, 'co...",0.227,0.705,0.068,0.7430
2,6.0,0.3933,"leg, recline, front, room, back, row, space, u...",solid experience start finish especially given...,5.0,0.7,"""A solid experience""",2019-04-01,Australia,"A solid experience from start to finish, espec...",...,NaN,NaN,NaN,NaN,solid experience start finish especially given...,"{'neg': 0.051, 'neu': 0.777, 'pos': 0.172, 'co...",0.172,0.777,0.051,0.8979
3,6.0,0.3516,"leg, recline, front, room, back, row, space, u...",discovered upon boarding four year old allocat...,0.0,0.1,"""my 4yr old and I allocated different seats""",2018-01-14,Australia,I discovered upon boarding that my four-year-o...,...,NaN,NaN,NaN,NaN,discovered upon boarding four year old allocat...,"{'neg': 0.161, 'neu': 0.726, 'pos': 0.113, 'co...",0.113,0.726,0.161,-0.5267
4,6.0,0.4102,"leg, recline, front, room, back, row, space, u...",got seat near emergency exit different standar...,1.0,0.7,"""space between was relatively good""",2017-08-24,Indonesia,"I got 14A seat, near emergency exit door. So t...",...,NaN,NaN,NaN,NaN,got seat near emergency exit different standar...,"{'neg': 0.157, 'neu': 0.66, 'pos': 0.183, 'com...",0.183,0.660,0.157,0.2960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,6.0,0.6278,"leg, recline, front, room, back, row, space, u...",took economy seat sgn hkk sfo last time ever f...,0.0,0.4,"""last time I ever fly United Economy""",2008-12-30,United States,Took the economy seat from SGN-HKK-SFO. It is ...,...,NaN,NaN,NaN,NaN,took economy seat sgn hkk sfo last time ever f...,"{'neg': 0.056, 'neu': 0.708, 'pos': 0.236, 'co...",0.236,0.708,0.056,0.9451
2590,6.0,0.3895,"leg, recline, front, room, back, row, space, u...",uncomfortable trip ever taken seat united airb...,0.0,0.3,"""one of the most uncomfortable trips""",2008-12-28,United States,This was one of the most uncomfortable trips I...,...,NaN,NaN,NaN,NaN,uncomfortable trip ever taken seat united airb...,"{'neg': 0.141, 'neu': 0.489, 'pos': 0.37, 'com...",0.370,0.489,0.141,0.4939
2591,4.0,0.4101,"good, comfortable, use, entertainment, really,...",aircraft clean well presented good finish seat...,0.0,0.9,"""clean and well presented""",2018-09-02,United Kingdom,Aircraft was clean and well presented with goo...,...,NaN,NaN,NaN,NaN,aircraft clean well presented good finish seat...,"{'neg': 0.0, 'neu': 0.67, 'pos': 0.33, 'compou...",0.330,0.670,0.000,0.9300
2592,6.0,0.3482,"leg, recline, front, room, back, row, space, u...",seated row return trip chose seat near emergen...,1.0,0.6,"""this row does not recline""",2018-04-19,United Arab Emirates,I was seated in rows 11 for both return trips....,...,NaN,NaN,NaN,NaN,seated row return trip chose seat near emergen...,"{'neg': 0.07, 'neu': 0.786, 'pos': 0.144, 'com...",0.144,0.786,0.070,0.4019
